In [ ]:
from google.colab import drive

drive.mount(r'/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torchtext
!pip install datasets
!pip install evaluate
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
import torch
import random
import numpy as np

seed = 1234
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [ ]:
from datasets import load_dataset

# train_dataset = load_dataset("ted_talks_iwslt", language_pair=("en", "fr"), year="2014", trust_remote_code=True, split='train')
# dev_dataset = load_dataset("ted_talks_iwslt", language_pair=("en", "fr"), year="2015", trust_remote_code=True, split='train')
# test_dataset = load_dataset("ted_talks_iwslt", language_pair=("en", "fr"), year="2016", trust_remote_code=True, split='train')

train_dataset, dev_dataset, test_dataset = load_dataset("ahazeemi/iwslt14-en-fr", split=["train", "validation", "test"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/179435 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/903 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3666 [00:00<?, ? examples/s]

In [ ]:
from datasets import Dataset
from datasets import concatenate_datasets

total_len = len(train_dataset) + len(dev_dataset) + len(test_dataset)
train_len = int(0.8 * total_len)
dev_len = int(0.1 * total_len)
test_len = total_len - train_len - dev_len

train_dataset = concatenate_datasets([Dataset.from_list(train_dataset['translation']),
                                          Dataset.from_list(dev_dataset[dev_len:]['translation']),
                                          Dataset.from_list(test_dataset[test_len:]['translation'])])

dev_dataset = Dataset.from_list(dev_dataset[:dev_len]['translation'])
test_dataset = Dataset.from_list(test_dataset[:test_len]['translation'])

In [ ]:
print(train_dataset)
print(dev_dataset)
print(test_dataset)

Dataset({
    features: ['id', 'translation', 'en', 'fr'],
    num_rows: 179435
})
Dataset({
    features: ['id', 'translation', 'en', 'fr'],
    num_rows: 903
})
Dataset({
    features: ['id', 'translation', 'en', 'fr'],
    num_rows: 3666
})


In [ ]:
import spacy

def tokenize_example(example, en_nlp, fr_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example['en'])][:max_length]
    fr_tokens = [token.text for token in fr_nlp.tokenizer(example['fr'])][:max_length]

    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        fr_tokens = [token.lower() for token in fr_tokens]

    en_tokens = [sos_token] + en_tokens + [eos_token]
    fr_tokens = [sos_token] + fr_tokens + [eos_token]

    return {"en_tokens": en_tokens, "fr_tokens": fr_tokens}

fr_nlp = spacy.load("fr_core_news_sm", disable=["tagger", "attribute_ruler", "lemmatizer"])
en_nlp = spacy.load("en_core_web_sm", disable=["tagger", "attribute_ruler", "lemmatizer"])

max_length  = 1000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "fr_nlp": fr_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)
dev_data = dev_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/179435 [00:00<?, ? examples/s]

Map:   0%|          | 0/903 [00:00<?, ? examples/s]

Map:   0%|          | 0/3666 [00:00<?, ? examples/s]

In [ ]:
train_data[0]

{'id': 'docid-1_segid-1',
 'translation': {'en': 'It can be a very complicated thing, the ocean.',
  'fr': "Ca peut être très compliqué, l'océan."},
 'en': 'It can be a very complicated thing, the ocean.',
 'fr': "Ca peut être très compliqué, l'océan.",
 'en_tokens': ['<sos>',
  'it',
  'can',
  'be',
  'a',
  'very',
  'complicated',
  'thing',
  ',',
  'the',
  'ocean',
  '.',
  '<eos>'],
 'fr_tokens': ['<sos>',
  'ca',
  'peut',
  'être',
  'très',
  'compliqué',
  ',',
  "l'",
  'océan',
  '.',
  '<eos>']}

In [ ]:
import torchtext

min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(train_data['en_tokens'], min_freq, specials=special_tokens)
fr_vocab = torchtext.vocab.build_vocab_from_iterator(train_data['fr_tokens'], min_freq, specials=special_tokens)

assert en_vocab[unk_token] == fr_vocab[unk_token]
assert en_vocab[pad_token] == fr_vocab[pad_token]

In [ ]:
unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

print(unk_index)
print(pad_index)

en_vocab.set_default_index(unk_index)
fr_vocab.set_default_index(unk_index)

0
1


In [ ]:
en_vocab["The"]

0

In [ ]:
en_vocab.get_itos()[0]

'<unk>'

In [ ]:
print(len(en_vocab))
print(len(fr_vocab))

31456
40486


In [ ]:
def numerical_example(example, en_vocab, fr_vocab):
    end_ids = en_vocab.lookup_indices(example['en_tokens'])
    fr_ids = fr_vocab.lookup_indices(example['fr_tokens'])

    return {"en_ids": end_ids, "fr_ids": fr_ids}

fn_kwargs = {"en_vocab": en_vocab, "fr_vocab": fr_vocab}
train_data = train_data.map(numerical_example, fn_kwargs=fn_kwargs)
dev_data = dev_data.map(numerical_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numerical_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/179435 [00:00<?, ? examples/s]

Map:   0%|          | 0/903 [00:00<?, ? examples/s]

Map:   0%|          | 0/3666 [00:00<?, ? examples/s]

In [ ]:
train_data[0]

{'id': 'docid-1_segid-1',
 'translation': {'en': 'It can be a very complicated thing, the ocean.',
  'fr': "Ca peut être très compliqué, l'océan."},
 'en': 'It can be a very complicated thing, the ocean.',
 'fr': "Ca peut être très compliqué, l'océan.",
 'en_tokens': ['<sos>',
  'it',
  'can',
  'be',
  'a',
  'very',
  'complicated',
  'thing',
  ',',
  'the',
  'ocean',
  '.',
  '<eos>'],
 'fr_tokens': ['<sos>',
  'ca',
  'peut',
  'être',
  'très',
  'compliqué',
  ',',
  "l'",
  'océan',
  '.',
  '<eos>'],
 'en_ids': [2, 14, 36, 39, 10, 57, 1026, 118, 4, 6, 556, 5, 3],
 'fr_ids': [2, 533, 81, 69, 67, 1691, 4, 14, 748, 5, 3]}

In [ ]:
en_vocab.lookup_tokens(train_data[0]["en_ids"])

['<sos>',
 'it',
 'can',
 'be',
 'a',
 'very',
 'complicated',
 'thing',
 ',',
 'the',
 'ocean',
 '.',
 '<eos>']

In [ ]:
data_type="torch"
format_columns = ["en_ids", "fr_ids"]

train_data = train_data.with_format(type=data_type, columns=format_columns,  output_all_columns=True)
dev_data = dev_data.with_format(type=data_type, columns=format_columns,  output_all_columns=True)
test_data = test_data.with_format(type=data_type, columns=format_columns,  output_all_columns=True)

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example['en_ids'] for example in batch]
        batch_fr_ids = [example['fr_ids'] for example in batch]
        batch_en_ids = pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_fr_ids = pad_sequence(batch_fr_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "fr_ids": batch_fr_ids,
        }

        return batch

    return collate_fn

def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = DataLoader(dataset=dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=shuffle)

    return data_loader

batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
dev_data_loader = get_data_loader(dev_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

In [ ]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers, dropout):
        super(Encoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, source):
        embedding = self.dropout(self.embedding(source))
        outputs, (hidden, cell) = self.rnn(embedding)

        return hidden, cell


class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, num_layers, dropout):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.output_dim = output_dim

        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        prediction = self.fc(output.squeeze(0))

        return prediction, hidden, cell


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hidden_dim == decoder.hidden_dim, "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.num_layers == decoder.num_layers, "Encoder and decoder must have equal number of layers!"

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = target.shape[1]
        target_len = target.shape[0]
        target_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(self.device)

        hidden, cell = self.encoder(source)
        input = target[0, :]

        for t in range(1, target_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            best_guess = output.argmax(1)
            input = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs

In [ ]:
input_dim = len(en_vocab)
output_dim = len(fr_vocab)
encoder_embedding_dim = 128 # 256
decoder_embedding_dim = 128 # 256
hidden_dim = 100 #512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(input_dim, encoder_embedding_dim, hidden_dim, n_layers, encoder_dropout)

decoder = Decoder(output_dim, decoder_embedding_dim, hidden_dim, n_layers, decoder_dropout)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(31456, 128)
    (rnn): LSTM(128, 100, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(40486, 128)
    (rnn): LSTM(128, 100, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=100, out_features=40486, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 13,643,262 trainable parameters


In [ ]:
def train_fn(model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        source = batch['en_ids'].to(device)
        target = batch['fr_ids'].to(device)

        optimizer.zero_grad()

        output = model(source, target, teacher_forcing_ratio)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        target = target[1:].view(-1)
        loss = criterion(output, target.long())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(data_loader)

def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            source = batch['en_ids'].to(device)
            target = batch['fr_ids'].to(device)

            output = model(source, target, 0) # turn off teacher forcing

            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            target = target[1:].view(-1)
            loss = criterion(output, target.long())
            epoch_loss += loss.item()


    return epoch_loss / len(data_loader)

In [ ]:
import tqdm

optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=0.01)
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)
epochs = 7
clip = 1.0
teacher_forcing_ratio = 0.5
best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(epochs)):
  train_loss = train_fn( model, train_data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device)

  valid_loss = evaluate_fn(model, dev_data_loader, criterion, device)

  if valid_loss < best_valid_loss:
      best_valid_loss = valid_loss
      torch.save(model.state_dict(), '/content/drive/MyDrive/data/tut1-model.pt')
  print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
  print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

NameError: name 'torch' is not defined

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/data/tut1-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

RuntimeError: Error(s) in loading state_dict for Seq2Seq:
	Missing key(s) in state_dict: "encoder.rnn.weight_ih_l0_reverse", "encoder.rnn.weight_hh_l0_reverse", "encoder.rnn.bias_ih_l0_reverse", "encoder.rnn.bias_hh_l0_reverse", "encoder.rnn.weight_ih_l1_reverse", "encoder.rnn.weight_hh_l1_reverse", "encoder.rnn.bias_ih_l1_reverse", "encoder.rnn.bias_hh_l1_reverse". 
	size mismatch for encoder.rnn.weight_ih_l1: copying a param with shape torch.Size([2048, 512]) from checkpoint, the shape in current model is torch.Size([2048, 1024]).

In [ ]:
def translate_sentence(sentence, model, en_nlp, en_vocab, fr_vocab, lower, sos_token, eos_token, device, max_output_length=25):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in en_nlp.tokenizer(sentence)]

        if lower:
            tokens = [token.lower() for token in tokens]

        tokens = [sos_token] + tokens + [eos_token]
        ids = en_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = fr_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            input_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(input_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == fr_vocab[eos_token]:
                break

        tokens = fr_vocab.lookup_tokens(inputs)

    return tokens

In [ ]:
sentence = test_data[10]['en']
human_translation = test_data[10]['fr']

print(sentence)
print(human_translation)

translation = translate_sentence(sentence, model, en_nlp, en_vocab, fr_vocab, lower, sos_token, eos_token, device)
print(translation)

There are a few things that we all need: fresh air, water, food, shelter, love ... and a safe place to pee. For trans people who don't fit neatly into the gender binary, public restrooms are a major source of anxiety and the place where they are most likely to be questioned or harassed. In this poetically rhythmic talk, Ivan Coyote grapples with complex and intensely personal issues of gender identity and highlights the need for gender-neutral bathrooms in all public places.
Nous avons besoin de plusieurs choses : d'air, d'eau, de nourriture, d'un toit, d'amour... et un endroit sûr pour faire pipi. Pour les transgenres, qui ne collent pas aux genres binaires, les toilettes publiques sont une grande source d'angoisse et le lieu où ils sont le plus interpellés ou harcelés. Dans son exposé aux allures poétiques, Ivan Coyote aborde les questions complexes et très personnelles d'identité sexuelle et met en lumière la nécessité de toilettes « neutres » dans tous les lieux publics.


RuntimeError: Expected hidden[0] size (2, 1, 512), got [4, 1, 512]

In [ ]:
import tqdm

translations = [
    translate_sentence(
        example["en"],
        model,
        en_nlp,
        en_vocab,
        fr_vocab,
        lower,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm.tqdm(test_data)
]

  0%|          | 0/1076 [00:00<?, ?it/s]

RuntimeError: Expected hidden[0] size (2, 1, 512), got [4, 1, 512]

In [ ]:
import evaluate

bleu = evaluate.load("bleu")
predictions = [" ".join(translation[1:-1]) for translation in translations]
references = [[example["fr"]] for example in test_data]

def get_tokenizer_fn(nlp, lower):
    def tokenizer_fn(s):
        tokens = [token.text for token in nlp.tokenizer(s)]
        if lower:
            tokens = [token.lower() for token in tokens]
        return tokens

    return tokenizer_fn

tokenizer_fn = get_tokenizer_fn(fr_nlp, lower)
tokenizer_fn(predictions[0]), tokenizer_fn(references[0][0])
results = bleu.compute(
    predictions=predictions, references=references, tokenizer=tokenizer_fn
)

results

{'bleu': 0.0,
 'precisions': [0.1386121897686983, 0.0, 0.0, 0.0],
 'brevity_penalty': 0.13213672995054163,
 'length_ratio': 0.33069679148365966,
 'translation_length': 17769,
 'reference_length': 53732}

In [ ]:
def beam_search(model, source, beam_width, max_length, sos_index, eos_index, fr_vocab, device):
    model.eval()

    candidates = [([sos_index], 1)]
    top_candidates = []

    source_tensor = torch.LongTensor(source).unsqueeze(1).to(device)

    with torch.no_grad():
        encoder_hidden, encoder_cell = model.encoder(source_tensor)

    decoder_hidden = encoder_hidden
    decoder_cell = encoder_cell

    for _ in range(max_length):
        current_candidates = []

        for _, (candidate, sentence_prob) in enumerate(candidates):
            last_token = candidate[-1]

            if last_token == eos_index:
                top_candidates.append(candidate)
                continue

            input_tensor = torch.LongTensor([last_token]).to(device)

            with torch.no_grad():
                output, decoder_hidden, decoder_cell = model.decoder(input_tensor, decoder_hidden, decoder_cell)

            topk_probs, topk_tokens = torch.topk(output.squeeze(0), beam_width)

            for i in range(beam_width):
                new_candidate = candidate + [topk_tokens[i].item()]

                current_candidates.append((new_candidate, topk_probs[i].item() * sentence_prob))

        candidates = [(candidate, prob) for candidate, prob in sorted(current_candidates, key=lambda x: x[1], reverse=True)[:beam_width]]

        if all(candidate[-1] == eos_index for candidate in candidates):
            break

    if len(candidates) == 0:
        return fr_vocab.lookup_tokens([sos_index])

    result, _ = candidates[0]
    return fr_vocab.lookup_tokens(result)

In [ ]:
source_sequence = ["hello"]
source_sequence_indices = en_vocab.lookup_indices(source_sequence)
beam_width = 2
max_length = 10
sos_index = fr_vocab.lookup_indices([sos_token])[0]
eos_index = fr_vocab.lookup_indices([eos_token])[0]

generated_sequence = beam_search(model, source_sequence_indices, beam_width, max_length, sos_index, eos_index, fr_vocab, device)


print(generated_sequence)

['<sos>', ':', ':', ':', ':', ',', ',', 'de', 'de', 'de', ',']


In [ ]:
import matplotlib.pyplot as plt

beam_widths = list(range(1, 101))

bleu_scores = []
evaluate_bleu = evaluate.load("bleu")

for beam_width in tqdm.tqdm(beam_widths):
    translations = [beam_search(model, source, beam_width, 50, sos_index, eos_index, fr_vocab, device) for source in test_data['en_ids']]
    predictions = [" ".join(translation[1:-1]) for translation in translations]
    references = [[example["fr"]] for example in test_data]
    tokenizer_fn = get_tokenizer_fn(fr_nlp, lower)
    tokenizer_fn(predictions[0]), tokenizer_fn(references[0][0])
    results = evaluate_bleu.compute(
        predictions=predictions, references=references, tokenizer=tokenizer_fn
    )

    bleu_scores.append(results)

plt.plot(beam_widths, bleu_scores)
plt.xlabel('Beam Width')
plt.ylabel('BLEU Score')
plt.title('Change in BLEU Score with Different Beam Widths')
plt.grid(True)
plt.show()


  9%|▉         | 9/100 [5:05:56<74:10:52, 2934.65s/it]

In [ ]:
from transformers import AutoTokenizer
from collections import defaultdict

def compute_word_freq(corpus, tokenizer):
    word_freqs = defaultdict()
    for text in corpus:
        words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
        new_words = [word for word, offset in words_with_offsets]

        for word in new_words:
            word_freqs[word] += 1

    return word_freqs

def compute_base_vocab(word_freqs):
    alphabet = []

    for word in word_freqs.keys():
        for letter in word:
            if letter not in alphabet:
                alphabet.append(letter)

    alphabet.sort()
    vocab = ["<eos>", "<sos>"] + alphabet.copy()

    return vocab

def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue

        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq

    return pair_freqs


tokenizer = AutoTokenizer.from_pretrained("gpt2")
word_freqs = compute_word_freq(train_data['en_tokens'])
vocab = compute_base_vocab(word_freqs)
splits = {word: [c for c in word] for word in word_freqs.keys()}
pair_freqs = compute_pair_freqs(splits)
